In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import SGD
from pyimagesearch import config
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import optimizers
from imutils import paths
import pandas as pd
import numpy as np
import tensorflow
import random
import shutil
import sys
import cv2
import sys
import os


import matplotlib
matplotlib.use("Agg")

In [2]:
conv_base = VGG16(weights = "imagenet",
                  include_top = False,
                  input_shape=(64, 64, 3))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [3]:
# grab the paths to all input images in the original input directory
# and shuffle them
imagePaths = list(paths.list_images(config.ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)

In [4]:
# compute the training and testing split
i = int(len(imagePaths) * config.TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]

# we'll be using part of the training data for validation
i = int(len(trainPaths) * config.VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]

In [5]:
# define the datasets that we'll be building
datasets = [
        ("training", trainPaths, config.TRAIN_PATH),
        ("validation", valPaths, config.VAL_PATH),
        ("testing", testPaths, config.TEST_PATH)
]

In [8]:
# loop over the datasets

for (dType, imagePaths, baseOutput) in datasets:

    # show which data split we are creating
    print("[INFO] building '{}' split".format(dType))

    # if the output base output directory does not exist, create it
    if not os.path.exists(baseOutput):
        print("[INFO] 'creating {}' directory".format(baseOutput))
        os.makedirs(baseOutput)

    # loop over the input image paths
    for inputPath in imagePaths:
        # extract the filename of the input image along with its
        # corresponding class label
        filename = inputPath.split(os.path.sep)[-1]
        label = inputPath.split(os.path.sep)[-2]

        # build the path to the label directory
        labelPath = os.path.sep.join([baseOutput, label])

        # if the label output directory does not exist, create it
        if not os.path.exists(labelPath):
            print("[INFO] 'creating {}' directory".format(labelPath))
            os.makedirs(labelPath)

        # construct the path to the destination image and then copy
        # the image itself
        p = os.path.sep.join([labelPath, filename])
        shutil.copy2(inputPath, p)


[INFO] building 'training' split
[INFO] building 'validation' split
[INFO] building 'testing' split


In [6]:
from keras.preprocessing.image import ImageDataGenerator

# initialize the training training data augmentation object
trainAug = ImageDataGenerator(
    rescale=1 / 255.0,
    rotation_range=40,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    #fill_mode="nearest"
    )

valAug = ImageDataGenerator(rescale=1 / 255.0)

testAug = ImageDataGenerator(rescale=1 / 255.0)


In [7]:
# define the total number of epochs to train for along with the
# initial learning rate and batch size
NUM_EPOCHS = 50
batch_size = 32

totalTrain = len(list(paths.list_images(config.TRAIN_PATH)))
totalVal = len(list(paths.list_images(config.VAL_PATH)))
totalTest = len(list(paths.list_images(config.TEST_PATH)))

In [8]:
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 2, 2, 512))
    labels = np.zeros(shape=(sample_count, 2))

    generator = ImageDataGenerator(
        rescale=1./255).flow_from_directory(
        directory,
        target_size=(64, 64),
        batch_size = batch_size, 
        color_mode="rgb",
        shuffle=False,
        class_mode='categorical')

    i = 0

    print('Entering for loop...');

    
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels, generator

In [9]:
train_features, train_labels, train_generator = extract_features(config.TRAIN_PATH,totalTrain)

Found 19842 images belonging to 2 classes.
Entering for loop...


In [10]:
validation_features, validation_labels, validation_generator= extract_features(config.VAL_PATH, totalVal)

Found 2204 images belonging to 2 classes.
Entering for loop...


In [11]:
test_features, test_labels, test_generator = extract_features(config.TEST_PATH, totalTest)

Found 5512 images belonging to 2 classes.
Entering for loop...


In [12]:
train_features = np.reshape(train_features, (totalTrain, 2 * 2 * 512))
validation_features = np.reshape(validation_features, (totalVal, 2 * 2 * 512))
test_features = np.reshape(test_features, (totalTest, 2 * 2 * 512))

In [13]:
np.save('train_features', train_features)

train_features = np.load('train_features.npy')

In [14]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Dense(2048, activation='relu', input_dim=2 * 2 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu', input_dim=2 * 2 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu', input_dim=2 * 2 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               65664     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 2

In [16]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer=tensorflow.keras.optimizers.Adam(learning_rate=1e-4),
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=50,
                    batch_size=32,
                    validation_data=(validation_features, validation_labels))

Epoch 1/50
621/621 [==============================] - 20s 31ms/step - loss: 0.3956 - acc: 0.8162 - val_loss: 0.2308 - val_acc: 0.9129
Epoch 2/50
621/621 [==============================] - 20s 33ms/step - loss: 0.2554 - acc: 0.8968 - val_loss: 0.1991 - val_acc: 0.9256
Epoch 3/50
621/621 [==============================] - 20s 32ms/step - loss: 0.2237 - acc: 0.9142 - val_loss: 0.1887 - val_acc: 0.9301
Epoch 4/50
621/621 [==============================] - 21s 33ms/step - loss: 0.2080 - acc: 0.9196 - val_loss: 0.1796 - val_acc: 0.9338
Epoch 5/50
621/621 [==============================] - 20s 32ms/step - loss: 0.1971 - acc: 0.9235 - val_loss: 0.1768 - val_acc: 0.9347
Epoch 6/50
621/621 [==============================] - 20s 32ms/step - loss: 0.1887 - acc: 0.9284 - val_loss: 0.1849 - val_acc: 0.9297
Epoch 7/50
621/621 [==============================] - 20s 33ms/step - loss: 0.1851 - acc: 0.9303 - val_loss: 0.1802 - val_acc: 0.9297
Epoch 8/50
621/621 [==============================] - 22s 35ms

In [18]:
model.evaluate(test_features,test_labels)

173/173 [==============================] - 1s 8ms/step - loss: 0.2005 - acc: 0.9454


[0.2005234807729721, 0.9453918933868408]